In [ ]:
pip install mrjob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 KB 7.6 MB/s eta 0:00:00


**ENCODING CSV FILE**

In [ ]:
import chardet
import pandas as pd

with open('Movies.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result


{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}

In [ ]:
movie = pd.read_csv('Movies.csv',encoding='ISO-8859-1')
movie.head()

,Year;Length;Title;Genre;Actor;Actress;Director;Popularity;Awards;*Image,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,INT;INT;STRING;CAT;CAT;CAT;CAT;INT;BOOL;STRING,NaN,NaN,NaN,NaN,NaN
1,1990;111;Tie Me Up! Tie Me Down!;Comedy;Banderas,Antonio;Abril,Victoria;Almodóvar,Pedro;68;No;NicholasCage.png,NaN,NaN
2,1991;113;High Heels;Comedy;Bosé,Miguel;Abril,Victoria;Almodóvar,Pedro;68;No;NicholasCage.png,NaN,NaN
3,1983;104;Dead Zone,The;Horror;Walken,Christopher;Adams,Brooke;Cronenberg,David;79;No;NicholasCage.png,NaN
4,1979;122;Cuba;Action;Connery,Sean;Adams,Brooke;Lester,Richard;6;No;seanConnery.png,NaN,NaN


#**QUESTION 1**

**1. Find the total number of movies of each genre. Consider only the movies released after 1970 and
have a length greater than 75 minutes.**

In [ ]:
%%file movies_count.py
from mrjob.job import MRJob
import csv
import io

class Movie_Genre_Count(MRJob):

   def mapper(self, _, line):
    row = line.strip().split(";")
    try:
        year, length, _, genre, _, _, _, _, _, _ = row
        if int(year) > 1970 and int(length) > 75:
            yield genre, 1
    except ValueError:
        pass

    def combiner(self, genre, counts):
        yield genre, sum(counts)

    def reducer(self, genre, counts):
        yield genre, sum(counts)

if __name__ == '__main__':
   io.open('Movies.csv', encoding='ISO-8859-1')
   Movie_Genre_Count.run()

Overwriting movies_count.py


In [ ]:
!python 'movies_count.py' 'Movies.csv'

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/movies_count.root.20230327.170907.077903
Running step 1 of 1...
job output is in /tmp/movies_count.root.20230327.170907.077903/output
Streaming final output from /tmp/movies_count.root.20230327.170907.077903/output...
"Drama"	1
"Drama"	1
"Drama"	1
"Drama"	1
"Drama"	1
"Drama"	1
"Comedy"	1
"Science Fiction"	1
"Science Fiction"	1
"Science Fiction"	1
"Drama"	1
"Comedy"	1
"Drama"	1
"Drama"	1
"Mystery"	1
"Drama"	1
"Comedy"	1
"Action"	1
"Mystery"	1
"Mystery"	1
"Mystery"	1
"Comedy"	1
"Mystery"	1
"Drama"	1
"Comedy"	1
"Drama"	1
"Comedy"	1
"Comedy"	1
"Comedy"	1
"Comedy"	1
"Action"	1
"Drama"	1
"Drama"	1
"Drama"	1
"Comedy"	1
"Drama"	1
"Drama"	1
"Drama"	1
"Comedy"	1
"Comedy"	1
"Drama"	1
"Drama"	1
"Western"	1
"Drama"	1
"Drama"	1
"Comedy"	1
"Mystery"	1
"Horror"	1
"Comedy"	1
"Action"	1
"Action"	1
"Action"	1
"Drama"	1
"Comedy"	1
"Drama"	1
"Horror"	1
"Drama"	1
"Comedy"	1
"Action"	1
"Dr

**2. For each actress, find the movie she acted in. You must print the movie's names and the movie's
year.**

In [ ]:
%%file movies_count.py
from mrjob.job import MRJob
import csv
import io

class ActressMovieList(MRJob):

    def mapper(self, _, line):
        if not line.startswith('Year'):
            year, length, title, genre, actor, actress, director, popularity, awards, image = line.split(';')
            yield actress, (title, year)

    def reducer(self, actress, movies):
        movies_list = list(movies)
        sorted_movies = sorted(movies_list, key=lambda x: x[1])
        yield actress, sorted_movies

if __name__ == '__main__':
  io.open('Movies.csv', encoding='ISO-8859-1')
  ActressMovieList.run()


Overwriting movies_count.py


In [ ]:
!python 'movies_count.py' 'Movies.csv'

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/movies_count.root.20230327.170907.421894
Running step 1 of 1...
job output is in /tmp/movies_count.root.20230327.170907.421894/output
Streaming final output from /tmp/movies_count.root.20230327.170907.421894/output...
"Bates, Kathy"	[["Misery", "1990"]]
"Baude, Anna-Lisa"	[["Crisis", "1946"]]
"Bauer, Belinda"	[["Samson & Delilah", "1984"], ["Act of Piracy", "1990"]]
"Beals, Jennifer"	[["Split Decisions", "1988"], ["Vampire's Kiss", "1989"]]
"Beck, Kimberly"	[["Nightmare at Noon", "1988"]]
"Bedelia, Bonnie"	[["Presumed Innocent", "1990"]]
"Beecher, Janet"	[["Reap the Wild Wind", "1942"]]
"Belford, Christine"	[["Pocket Money", "1972"]]
"Beller, Kathleen"	[["Mary White", "1977"]]
"Benatar, Pat"	[["Catch a Rising Star, Tenth Anniversary", "1982"]]
"Bening, Annette"	[["Guilty by Suspicion", "1990"]]
"Bennett, Joan"	[["Scarlet Street", "1945"], ["Secret Beyond the Door", "

**3. Find the 25 most popular movies released during 1950-1990**

In [ ]:
%%file movies_count.py
from mrjob.job import MRJob
import csv
import io
from collections import defaultdict

class PopularMovies(MRJob):

  def mapper(self, _, line):
    if "Year;Length;Title;Genre;Actor;Actress;Director;Popularity;Awards;*Image" in line:
        return

    row = line.strip().split(";")
    try:
        year = int(row[0])
    except ValueError:
        return

    if 1950 <= year <= 1990 and row[7]:
        yield (row[2], float(row[7]))


    def reducer(self, key, values):
        yield None, (key, sum(values))

    def mapper_sort(self, _, line):
        movie, popularity = line.strip().split("\t")
        yield float(popularity), movie

    def reducer_sort(self, key, values):
        for movie in values:
            yield movie, key

if __name__ == '__main__':
    io.open('Movies.csv', encoding='ISO-8859-1')
    PopularMovies.run()

Overwriting movies_count.py


In [ ]:
!python 'movies_count.py' 'Movies.csv'

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/movies_count.root.20230327.175239.754388
Running step 1 of 1...
job output is in /tmp/movies_count.root.20230327.175239.754388/output
Streaming final output from /tmp/movies_count.root.20230327.175239.754388/output...
"Places in the Heart"	83.0
"Heroes"	17.0
"Absence of Malice"	76.0
"Norma Rae"	64.0
"Steel Magnolias"	66.0
"Burbs, The"	42.0
"Empire Strikes Back, The"	33.0
"Star Wars"	44.0
"Return of the Jedi"	4.0
"Slightly Scarlet"	52.0
"Gunfight at the OK Corral"	84.0
"Bloodsucking Pharaohs in Pittsburgh"	79.0
"Roma"	75.0
"China Syndrome, The"	43.0
"Morning After, The"	6.0
"Klute"	15.0
"Electric Horseman, The"	34.0
"Cat Ballou"	62.0
"Stacey!"	31.0
"Stripped to Kill II, Live Girls"	80.0
"Rain Killer, The"	10.0
"Valley Girl"	30.0
"Stealing Home"	76.0
"Napoleon & Samantha"	33.0
"Five Corners"	88.0
"Blackboard Jungle, The"	66.0
"My Left Foot"	32.0
"Back to the Beach"	45.

**4. Find the five most popular movies of each actor.**

In [ ]:
%%file movies_count.py
from mrjob.job import MRJob
import csv
import io
from heapq import nlargest


class ActorPopMovies(MRJob):

    def mapper(self, _, row):
        if row.startswith('Year'):
            return

        movie = row.split(';')

        try:
            popularity = float(movie[7])
        except ValueError:
            return

        actors = [a.strip() for a in movie[4].split(',')]
        title = movie[2]

        for actor in actors:
            yield actor, (popularity, title)

    def reducer(self, actor, values):
        top_movies = nlargest(5, values, key=lambda x: x[0])
        yield actor, top_movies


if __name__ == '__main__':
    io.open('Movies.csv', encoding='ISO-8859-1')
    ActorPopMovies.run()






Overwriting movies_count.py


In [ ]:
!python 'movies_count.py' 'Movies.csv'

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/movies_count.root.20230327.175701.962325
Running step 1 of 1...
job output is in /tmp/movies_count.root.20230327.175701.962325/output
Streaming final output from /tmp/movies_count.root.20230327.175701.962325/output...
"Eban"	[[69.0, "Double Edge"]]
"Ed"	[[83.0, "Places in the Heart"], [80.0, "Stripped to Kill II, Live Girls"], [43.0, "She-Devil"], [22.0, "Code Name, Emerald"], [2.0, "Mary White"]]
"Eddie"	[[86.0, "Golden Child, The"], [80.0, "Check Your Guns"], [61.0, "Hawk of Powder River"], [60.0, "Ladies of the Chorus"], [56.0, "Best of Eddie Murphy, Saturday Night Live, The"]]
"Edvin"	[[72.0, "Count of Old Town, The"], [26.0, "Only One Night"]]
"Edward G."	[[87.0, "Destroyer"], [80.0, "Scarlet Street"]]
"Edward James"	[[19.0, "Stand & Deliver"]]
"Edward"	[[82.0, "Butterflies Are Free"]]
"Edwards"	[[10.0, "Going for the Gold"]]
"Eklund"	[[29.0, "Port of Call"]]
"E

**5. Find the average length of the movies of each genre.**

In [ ]:
%%file movies_count.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
import io

class MovieGenreLength(MRJob):

    def mapper(self, _, line):
        movie = line.split(";")
        try:
            length = int(movie[1])
            genre = movie[3]
            yield genre, (length, 1)
        except ValueError:
            pass

    def combiner(self, genre, values):
        total_length = 0
        count = 0
        for length, movie_count in values:
            total_length += length * movie_count
            count += movie_count
        yield genre, (total_length, count)

    def reducer(self, genre, values):
        total_length = 0
        count = 0
        for length, movie_count in values:
            total_length += length
            count += movie_count
        yield genre, total_length / count

    def steps(self):
        return [
            MRStep(mapper=self.mapper, combiner=self.combiner, reducer=self.reducer)
        ]

if __name__ == '__main__':
  io.open('Movies.csv', encoding='ISO-8859-1')
  MovieGenreLength.run()


Overwriting movies_count.py


In [ ]:
!python 'movies_count.py' 'Movies.csv'

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/movies_count.root.20230327.170908.819481
Running step 1 of 1...
job output is in /tmp/movies_count.root.20230327.170908.819481/output
Streaming final output from /tmp/movies_count.root.20230327.170908.819481/output...
"Crime"	66.0
"Drama"	113.30455259026688
"Fantasy"	102.0
"Horror"	93.92727272727272
"Music"	100.48780487804878
"Mystery"	103.00990099009901
"Romance"	127.0
"Science Fiction"	106.47368421052632
"Short"	40.0
"War"	116.90625
""	120.5
"Action"	104.5
"Adventure"	119.0
"Comedy"	96.50540540540541
"Western"	93.0091743119266
"Westerns"	124.8
Removing temp directory /tmp/movies_count.root.20230327.170908.819481...


**6. Find the lead actor and lead actress pair who has acted in more than one movie together.**

In [ ]:
%%file movies_count.py
from mrjob.job import MRJob
from itertools import combinations
import csv
import io

class LeadActorActressPair(MRJob):

    def mapper(self, _, line):
        fields = line.strip().split(';')
        actors = [a.strip() for a in fields[4].split(',')]
        lead_actor, lead_actress = None, None
        if 'Actor' in fields[3] and 'Actress' in fields[3]:
            for actor in actors:
                if 'Actor' in fields[3]:
                    lead_actor = actor
                elif 'Actress' in fields[3]:
                    lead_actress = actor
            yield (lead_actor, lead_actress), fields[2]

    def reducer(self, key, values):
        movies = list(values)
        if len(movies) > 1:
            yield len(movies), key

if __name__ == '__main__':
    io.open('Movies.csv', encoding='ISO-8859-1')
    LeadActorActressPair.run()

#no output cuz there is no lead actor or actress pair who worked together in more than one movie

Overwriting movies_count.py


In [ ]:
!python 'movies_count.py' 'Movies.csv'

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/movies_count.root.20230327.181231.556934
Running step 1 of 1...
job output is in /tmp/movies_count.root.20230327.181231.556934/output
Streaming final output from /tmp/movies_count.root.20230327.181231.556934/output...
Removing temp directory /tmp/movies_count.root.20230327.181231.556934...


**7. Find the names of directors who directed movies of both the 'Mystery' and 'Drama' Genre.**

In [ ]:
%%file movies_count.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
import io


class DramaMysteryDirectors(MRJob):

    def mapper(self, _, line):
        if not line.startswith('Year'):
            year, length, title, genre, actor, actress, director, popularity, awards, image = line.split(';')
            yield director, (genre)

    def reducer(self, director, genres):
        genres_list = list(genres)
        if 'Drama' in genres_list and 'Mystery' in genres_list:
            yield director , None

if __name__ == '__main__':
   io.open('Movies.csv', encoding='ISO-8859-1')
   DramaMysteryDirectors.run()




Overwriting movies_count.py


In [ ]:
!python 'movies_count.py' 'Movies.csv'

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/movies_count.root.20230327.171346.457041
Running step 1 of 1...
job output is in /tmp/movies_count.root.20230327.171346.457041/output
Streaming final output from /tmp/movies_count.root.20230327.171346.457041/output...
"Demme, Jonathan"	null
"Eastwood, Clint"	null
"Foley, James"	null
"Hitchcock, Alfred"	null
"Kagan, Jeremy Paul"	null
"Kubrick, Stanley"	null
"Kurosawa, Akira"	null
"Lang, Fritz"	null
"Levinson, Barry"	null
"Locke, Sondra"	null
"Lumet, Sidney"	null
"Lynch, David"	null
"Lyne, Adrian"	null
"Mackenzie, John"	null
"Pakula, Alan J."	null
"Polanski, Roman"	null
""	null
"Aldrich, Robert"	null
"Attenborough, Richard"	null
"Avildsen, John G."	null
"Benton, Robert"	null
"Cimino, Michael"	null
"Rafelson, Bob"	null
"Ross, Herbert"	null
"Scorsese, Martin"	null
"Spielberg, Steven"	null
"Stern, Sandor"	null
"Stern, Steven Hilliard"	null
"Taylor, Jud"	null
"Welles, Orso

**8. Find each director, find the average, max and min ranking of his movies.**

In [ ]:
%%file movies_count.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
import io

class DirectorsRankings(MRJob):

    def mapper(self, _, line):
        fields = line.split(';')
        if len(fields) != 10:
            return

        year, length, title, _, _, director, _, ranking, _, _ = fields

        if not ranking.isnumeric():
            return
        yield director, (float(ranking), 1)

    def combiner(self, director, values):
        total_ranking = 0
        count = 0
        for ranking, n in values:
            total_ranking += ranking
            count += n
        yield director, (total_ranking, count)

    def reducer(self, director, values):
        total_ranking = 0
        count = 0
        max_ranking = -1
        min_ranking = 11
        for ranking, n in values:
            total_ranking += ranking
            count += n
            max_ranking = max(max_ranking, ranking)
            min_ranking = min(min_ranking, ranking)
        average_ranking = total_ranking / count
        yield director, (average_ranking, max_ranking, min_ranking)

    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   combiner=self.combiner,
                   reducer=self.reducer)
        ]

if __name__ == '__main__':
    io.open('Movies.csv', encoding='ISO-8859-1')
    DirectorsRankings.run()


Overwriting movies_count.py


In [ ]:
!python 'movies_count.py' 'Movies.csv'

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/movies_count.root.20230327.171934.944521
Running step 1 of 1...
job output is in /tmp/movies_count.root.20230327.171934.944521/output
Streaming final output from /tmp/movies_count.root.20230327.171934.944521/output...
"Davis, Bette"	[32.25, 129.0, 11]
"Davis, Geena"	[28.5, 57.0, 11]
"Davis, Sammi"	[34.5, 69.0, 11]
"Day, Doris"	[15.0, 15.0, 11]
"Day, Josette"	[14.0, 14.0, 11]
"Day, Laraine"	[61.0, 61.0, 11]
"De Havilland, Olivia"	[81.0, 81.0, 11]
"DeMornay, Rebecca"	[28.0, 28.0, 11]
"DeSoto, Rosana"	[12.0, 24.0, 11]
"Deakins, Lucy"	[25.0, 25.0, 11]
"Dean, Isabel"	[4.0, 4.0, 4.0]
"Dee, Frances"	[36.0, 36.0, 11]
"Dee, Ruby"	[19.0, 38.0, 11]
"Degermark, Pia"	[28.0, 28.0, 11]
"Delaney, Cassandra"	[20.0, 40.0, 11]
"Demongeot, Mylene"	[83.0, 83.0, 11]
"Deneuve, Catherine"	[47.0, 282.0, 11]
"Depardieu, Elizabeth"	[87.0, 87.0, 11]
"Dern, Laura"	[46.0, 92.0, 11]
"Dey, Susan"	[

# **QUESTION 2**

**In this example, you will use the given "Movies.csv" data file. Your task is to write an efficient MAP-Reduce
code for finding movies (released after 1970) that might be similar or related to a movie made in the
1960's (1960-1969).
To keep things simple, for comparing two movies, we will use the following fields: Genre, director, lead
actor, and lead actress.
So, we can say two movies are similar if they have the same genre, director, and same lead actor or actress.**

In [ ]:
%%file movies_count.py
from mrjob.job import MRJob
import csv
import io

class SimilarMovies(MRJob):
    def mapper(self, _, line):
    # Skip the first row (column headers)
       if "Year" in line:
          return

       fields = line.strip().split(";")
       year_str = fields[0]
       if not year_str.isnumeric():
           return
       year = int(year_str)
       if year > 1970:
        return

       key = (fields[3], fields[6], fields[4], fields[5])

       yield key, fields[2]



    def reducer(self, key, values):
        movies = set(values)
        if len(movies) > 1:
            yield key, sorted(movies)

if __name__ == '__main__':
  io.open('Movies.csv', encoding='ISO-8859-1')
  SimilarMovies.run()

Overwriting movies_count.py


In [ ]:
!python 'movies_count.py' 'Movies.csv'

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/movies_count.root.20230327.173428.915567
Running step 1 of 1...
job output is in /tmp/movies_count.root.20230327.173428.915567/output
Streaming final output from /tmp/movies_count.root.20230327.173428.915567/output...
["Drama", "Bergman, Ingmar", "Bj\u00f6rnstrand, Gunnar", "Andersson, Harriet"]	["Dreams", "Naked Night, The", "Through a Glass Darkly"]
["Drama", "Bergman, Ingmar", "Sydow, Max von", "Ullman, Liv"]	["Hour of the Wolf", "Passion of Anna, The"]
["Drama", "Brown, Clarence", "Gilbert, John", "Garbo, Greta"]	["Flesh & the Devil, The", "Woman of Affairs"]
["Drama", "Kurosawa, Akira", "Mifune, Toshiro", ""]	["Bad Sleep Well, The", "Idiot, The", "Rashomon", "Seven Samurai", "Throne of Blood"]
["Drama", "Lang, Fritz", "Klein-Rogge, Rudolf", ""]	["Spies", "Testament of Dr. Mabuse, The"]
["Drama", "Lelouch, Claude", "Trintignant, Jean-Louis", "Aimee, Anouk"]	["A M